In [ ]:
import os
import sys
import pandas as pd
from src import settings
from src.utils import fileio


def find(TREATMENT, SEARCH_ANGLE, SEARCH_DISTANCE, angle_tolerance, distance_tolerance, num_of_samples):

    counter = 0
    angles = fileio.load_files_from_folder(os.path.join(settings.OUTPUT_DIR, "0_1_2_angles_matrix", TREATMENT))
    distances = fileio.load_files_from_folder(os.path.join(settings.OUTPUT_DIR, "0_1_1_distances_matrix", TREATMENT))
    for angles_tuple, distances_tuple in zip(angles.items(), distances.items()):
        angles_name, angles_path = angles_tuple
        distances_name, distances_path = distances_tuple

        if angles_name != distances_name:
            sys.exit()

        df_angles = pd.read_csv(angles_path, index_col=0)
        df_distances = pd.read_csv(distances_path, index_col=0)

        for angles_col, distances_col in zip(df_angles.columns, df_distances.columns):
            if angles_col != distances_col:
                sys.exit()

            node_1, node_2 = [col.replace(".csv", "") for col in angles_col.split(" ")]

            angles_col_inverse = angles_col.split(" ")
            angles_col_inverse = angles_col_inverse[1] + " " + angles_col_inverse[0]

            df1 = pd.concat([df_angles[angles_col], df_distances[angles_col]], axis=1, keys=["angle", "distance"])

            mask1 = (
                (df1["angle"] >= SEARCH_ANGLE - angle_tolerance)
                & (df1["angle"] <= SEARCH_ANGLE + angle_tolerance)
                & (df1["distance"] >= SEARCH_DISTANCE - distance_tolerance)
                & (df1["distance"] <= SEARCH_DISTANCE + distance_tolerance)
            )

            df2 = pd.concat(
                [df_angles[angles_col_inverse], df_distances[angles_col_inverse]], axis=1, keys=["angle", "distance"]
            )
            mask2 = (
                (df2["angle"] >= SEARCH_ANGLE - angle_tolerance)
                & (df2["angle"] <= SEARCH_ANGLE + angle_tolerance)
                & (df2["distance"] >= SEARCH_DISTANCE - distance_tolerance)
                & (df2["distance"] <= SEARCH_DISTANCE + distance_tolerance)
            )

            if mask1.any() and mask2.any():
                # found_row = df[mask].index[0]

                df1 = df1[mask1].iloc[:5, :]
                df2 = df2[mask2].iloc[:5, :]
                # print(df1)
                # print(df2)
                merged_df = pd.merge(df1, df2, left_index=True, right_index=True)
                if len(merged_df):
                    print("=" * 88)
                    print(angles_name)
                    print(node_1, node_2)
                    print(merged_df)
                    print("=" * 88)
                    counter += 1

                    if counter == num_of_samples:
                        sys.exit()


TREATMENT = "CsCh"
SEARCH_ANGLE = 0
SEARCH_DISTANCE = 0.5
angle_tolerance = 10
distance_tolerance = 0.2
num_of_samples = 4

find(TREATMENT, SEARCH_ANGLE, SEARCH_DISTANCE, angle_tolerance, distance_tolerance, num_of_samples)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.